# Notebook Purpose

In this notebook, I implement RAG (Retrieval Augmented Generation) to build a program that allows me to ask a financial question about a company and get a response from ChatGPT that integrates new financial information into that response. 

ChatGPT may not have been trained on this new or existing financial information, but I retrieve financial information about the companies from Yahoo Finance summaries and Yahoo Finance news headlines to augment the response of the ChatGPT large language model (LLM). 

Even though ChatGPT was not trained on this data, it can incorporate this new data into its reasoning and response. That is the essence of how RAG works.

Retrieval Augmented Generation, in this context, allows me to connect external knowledge about the financial context of a company to the large language model in a meaningful way. This is done through embedding. Embedding converts the external knowledge into numerical vector representations, where texts with similar meanings are closer together in the vector space.

The first step in the sequence flow of the RAG embedding is to embed your new, external knowledge base. You gather your documents, split them into smaller chunks, and pass each chunk into an embedding model. The embedding model I use here is `all-MiniLM-L6-v2` from HuggingFace. Then I store the resulting vectors from the chunks of information into a vector database, which is `FAISS` here.

The next step in RAG is to embed the query from the user with the same model, here `all-MiniLM-L6-v2`.

The third step in the RAG sequence is to find similar chunks to the embedded query in the vector database, and retrieve the top `k` (here 3) most relevant chunks based on the information chunks that are closest in the vector space to the query.

Finally, you generate an answer from the LLM by combining the query and the retrieved context chunks, feeding it into the LLM, and then allowing the LLM to use the retrieved text to create more accurate and specific reponses.

# Code

In order to run this code, you will need to set your own secret OpenAI API Key equal to `OPENAI_API_KEY` in bash or your Windows command prompt (CMD). In the Windows CMD, this code is `set OPENAI_API_KEY=your-secret-key`.

In [1]:
#import libraries
import yfinance as yf
import feedparser
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import openai
import re
import os

C:\Users\mothn\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
#set your OpenAI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
#create a dictionary for company names to tickers
company_ticker_map = {
    "microsoft": "MSFT",
    "apple": "AAPL",
    "google": "GOOGL",
    "amazon": "AMZN",
    "meta": "META",
    "tesla": "TSLA",
    "bny": "BK"
    # Add more as needed
}

In [4]:
#create utility functions
def get_ticker_from_question(question):
    question_lower = question.lower()
    for name in company_ticker_map:
        if re.search(rf"{name}", question_lower):
            return company_ticker_map[name], name
    return None, None

In [5]:
def get_company_summary(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return info.get('longBusinessSummary', 'No summary available.')

In [6]:
def get_yahoo_finance_news(ticker):
    url = f"https://finance.yahoo.com/rss/headline?s={ticker}"
    feed = feedparser.parse(url)
    news_texts = [entry['title'] + ". " + entry['summary'] for entry in feed['entries']]
    return news_texts

In [7]:
def chunk_text(text, chunk_size=300):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

In [8]:
def build_rag_corpus(ticker, company_name):
    summary = get_company_summary(ticker)
    summary_chunks = chunk_text(summary)

    news_articles = get_yahoo_finance_news(ticker)
    news_chunks = [chunk_text(article) for article in news_articles]
    news_chunks = [item for sublist in news_chunks for item in sublist]

    return summary_chunks + news_chunks

In [9]:
#full RAG pipeline
def run_rag_with_news(question):
    ticker, company_name = get_ticker_from_question(question)
    if not ticker:
        return "Sorry, I couldn't identify a company in your question."

    corpus = build_rag_corpus(ticker, company_name)
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(corpus)

    index = faiss.IndexFlatL2(embeddings[0].shape[0])
    index.add(np.array(embeddings))

    def retrieve_context(q):
        q_embed = model.encode([q])
        D, I = index.search(np.array(q_embed), 3)
        return "\n".join([corpus[i] for i in I[0]])

    context = retrieve_context(question)
    prompt = f"Context:\n{context}\n\nQuestion: {question}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    return response['choices'][0]['message']['content']

In [10]:
#example search
if __name__ == "__main__":
    question = "What is BNY doing in AI this year?"
    answer = run_rag_with_news(question)
    print("\nGenerated Answer:\n", answer)

C:\Users\mothn\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Generated Answer:
 BNY Mellon has signed a multiyear deal with OpenAI, gaining access to cutting-edge AI tools such as Deep Research and advanced reasoning models. Additionally, BNY has made a minority investment in EquiLend, a company in the securities finance industry.
